<a href="https://colab.research.google.com/github/XinyuanLiao/ComplexNN/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install complexNN
%pip install torchsummaryX

  Preparing metadata (setup.py) ... done
  Created wheel for complexNN: filename=complexNN-0.4.2-py3-none-any.whl size=9656 sha256=17d1ee654cb4768217860f3747786273f78ba6d768e247d43f618ad16ba1d66b
  Stored in directory: /root/.cache/pip/wheels/82/8e/c2/8deeb36761f3a65ad250cf4757d2afb21f36887816fb7d44dd
Successfully built complexNN


# _Conv1d, Conv2d_

In [4]:
import torch
import torch.nn as nn
from torchsummaryX import summary
import complexNN.nn as cnn


class CNN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = cnn.cConv2d(in_channels, out_channels, padding='same')
        self.conv2 = cnn.cConv2d(out_channels, out_channels, padding='same')
        self.dropout = cnn.cDropout2d()
        self.maxpool = cnn.cMaxPool2d(kernel_size=3, stride=1)
        self.fc = cnn.cLinear(252*252, 10)
        self.ac = cnn.cRelu()

    def forward(self, x):
        x = self.dropout(self.ac(self.conv1(x)))
        x = self.maxpool(x)
        x = self.dropout(self.ac(self.conv2(x)))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], x.shape[1], -1)
        return self.fc(x)


if __name__ == '__main__':
    batch_size, in_channels, out_channels, seq_len = 1, 3, 16, 10
    H, W = 256, 256
    conv2d_tensor = torch.rand((batch_size, in_channels, H, W))
    convnn = CNN(in_channels, out_channels)
    print(convnn(conv2d_tensor).shape)
    summary(convnn, conv2d_tensor)

torch.Size([1, 16, 10])
             Kernel Shape       Output Shape   Params    Mult-Adds
Layer                                                             
0_conv1     [3, 16, 3, 3]  [1, 16, 256, 256]    432.0   28.311552M
1_ac                    -  [1, 16, 256, 256]        -            -
2_dropout               -  [1, 16, 256, 256]        -            -
3_maxpool               -  [1, 16, 254, 254]        -            -
4_conv2    [16, 16, 3, 3]  [1, 16, 254, 254]   2.304k  148.644864M
5_ac                    -  [1, 16, 254, 254]        -            -
6_dropout               -  [1, 16, 254, 254]        -            -
7_maxpool               -  [1, 16, 252, 252]        -            -
8_fc          [63504, 10]        [1, 16, 10]  635.05k      635.04k
-------------------------------------------------------------------
                           Totals
Total params             637.786k
Trainable params         637.786k
Non-trainable params          0.0
Mult-Adds             177.591456M


/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


# _MLP_

In [5]:
import torch
import complexNN.nn as cnn


if __name__ == '__main__':
    batch_size, input_size, hidden_size, output_size = 10, 10, 20, 15
    input_tensor = torch.rand((batch_size, input_size), dtype=torch.cfloat)
    mlp = cnn.cMLP(input_size, hidden_size, output_size, num_layers=3)
    out = mlp(input_tensor)
    print(out.shape)
    summary(mlp, input_tensor)

torch.Size([10, 15])
                          Kernel Shape Output Shape Params Mult-Adds
Layer                                                               
0_input_layer                 [10, 20]     [10, 20]  220.0     200.0
1_activation                         -     [10, 20]      -         -
2_hidden_layers.cLinear_0     [20, 20]     [10, 20]  420.0     400.0
3_activation                         -     [10, 20]      -         -
4_hidden_layers.cLinear_1     [20, 20]     [10, 20]  420.0     400.0
5_activation                         -     [10, 20]      -         -
6_output_layer                [20, 15]     [10, 15]  315.0     300.0
----------------------------------------------------------------------
                      Totals
Total params          1.375k
Trainable params      1.375k
Non-trainable params     0.0
Mult-Adds               1.3k


/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


# _RNN, GRU, LSTM_

In [6]:
import torch
import complexNN.nn as cnn


if __name__ == '__main__':
    batch_size, input_size, hidden_size, seq_len, num_layers = 10, 10, 20, 1, 3
    input_tensor = torch.rand((seq_len, batch_size, input_size), dtype=torch.cfloat)
    h0, c0 = torch.zeros((num_layers, batch_size, hidden_size)), torch.zeros((num_layers, batch_size, hidden_size))

    rnn = cnn.cRNN(input_size, hidden_size, num_layers)
    gru = cnn.cGRU(input_size, hidden_size, num_layers)
    lstm = cnn.cLSTM(input_size, hidden_size, num_layers)

    rnn_out, _ = rnn(input_tensor, h0)
    gru_out, _ = gru(input_tensor, h0)
    lstm_out, _ = lstm(input_tensor, (h0, c0))

    print(rnn_out.shape, gru_out.shape, lstm_out.shape)
    summary(rnn, input_tensor, h0)
    summary(gru, input_tensor, h0)
    summary(lstm, input_tensor, (h0, c0))

torch.Size([1, 10, 20]) torch.Size([1, 10, 20]) torch.Size([1, 10, 20])
                         Kernel Shape Output Shape  Params  Mult-Adds
Layer                                                                
0_rnn_cells.0.cLinear_Wx     [10, 20]     [10, 20]     220        200
1_rnn_cells.0.cLinear_Wh     [20, 20]     [10, 20]     420        400
2_rnn_cells.1.cLinear_Wx     [20, 20]     [10, 20]     420        400
3_rnn_cells.1.cLinear_Wh     [20, 20]     [10, 20]     420        400
4_rnn_cells.2.cLinear_Wx     [20, 20]     [10, 20]     420        400
5_rnn_cells.2.cLinear_Wh     [20, 20]     [10, 20]     420        400
---------------------------------------------------------------------
                      Totals
Total params            2320
Trainable params        2320
Non-trainable params       0
Mult-Adds               2200
                          Kernel Shape Output Shape  Params  Mult-Adds
Layer                                                                 
0_rnn_cells

/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()
/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()
/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is depreca